In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_fontja  # matplotlibで日本語を表示する
from config import ACCESS_TOKEN, USER_ID, API_VERSION, DESTINATION_FOLDER_PATH
from src import convert_timestamp_to_jst_formatted, fetch_my_posts
import requests
from PIL import Image
import io
from IPython.display import Image as IPyImage, display
import os

In [ ]:
def filter_df(
    search_term: str,
    df: pd.DataFrame
) -> pd.DataFrame:

    filtered_df = df[df['caption'].str.contains(search_term, na=False)]

    if len(filtered_df) != 1:
        print("該当の投稿が1件ではありません")
        return
    else:
        result_dict = filtered_df.to_dict(orient='records')[0]
        return result_dict

# イメージ取得用URLを作成
def make_img_url(
    permalink: str
) -> str:
    img_link = f"{permalink}media/?size=l"
    headers = {"User-Agent": ""}
    img_url = requests.get(img_link, headers=headers).url

    return img_url

# 画像をダウンロードする関数
def get_img(
    img_url: str,
    result_dict: dict,
    destination_folder_path: str = "/Users/st/workspace/instagram/imgs/",
    save_to_file: bool = True,       # 新しい引数: ファイルに保存するかどうか
    display_in_notebook: bool = True # Jupyter Notebookで表示するかどうか
) -> None:

    timestamp_jst = result_dict["timestamp_jst"]
    id = result_dict["id"]

    file_name = f"{destination_folder_path}/{timestamp_jst}_{id}.jpg"
    
    try:
        response = requests.get(img_url)
        response.raise_for_status() # HTTPエラーが発生した場合に例外を発生させる
        image_content = response.content

        # ファイルに保存する場合
        if save_to_file:
            # フォルダが存在しない場合は作成
            os.makedirs(destination_folder_path, exist_ok=True)
            with open(file_name, "wb") as f:
                f.write(image_content)
            print(f"画像を {file_name} にダウンロードしました。")
        else:
            print("画像をファイルに保存しませんでした。")

        # Jupyter Notebookで画像を表示する場合
        if display_in_notebook:
            try:
                # ダウンロードした画像データをメモリから開く
                image = Image.open(io.BytesIO(image_content))
                display(image) 
                print("Jupyter Notebookで画像を表示しました。")
            except Exception as e:
                print(f"Jupyter Notebookでの画像の表示中にエラーが発生しました: {e}")

    except requests.exceptions.RequestException as e:
        print(f"画像のダウンロード中にエラーが発生しました: {e}")
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")

In [ ]:
if 'results_df' not in locals() and 'results_df' not in globals():
    # データを読み込む
    results_df = fetch_my_posts(
        api_version=API_VERSION,
        user_id=USER_ID,
        access_token=ACCESS_TOKEN,
        num_iterations=150
    )
else:
    print("results_df は既にメモリに存在するため、データ読み込みをスキップしました。")

In [ ]:
serch_term = "最近部署内"

# serch_termのデータを抽出する
result_dict = filter_df(
    search_term=serch_term,
    df = results_df
)

# 画像のURLを取得する
img_url = make_img_url(result_dict["link"])

# 画像を指定したフォルダにダウンロードする
get_img(
    img_url=img_url,
    result_dict=result_dict,
    destination_folder_path=DESTINATION_FOLDER_PATH,
    save_to_file = True,
    display_in_notebook = True
)